## Desafio 2


Utilize a imagem da embalagem do iogurte, conforme a baixo, como alvo para obter suas características. Posteriormente, faça o mesmo buscando tais características em um vídeo promocional, para identificar o quanto da embalagem é exposto.

Aplique o algoritmo de extração de características de SURF e ORB, note a diferença entre ambos.
Para a correspondência de informações entre a embalagem e o vídeo, utilize o método de FLANN.

Após testes, estabeleça um limiar de pontos de correspondência e, por meio deste valor, informe se a embalagem é detectada ou não com um texto no próprio vídeo.

Embalagem de iogurte

<img src="imagens/coca-cola.jpg">

Captura de vídeo (apenas um frame)

<img src="imagens/coke-video.png">

In [2]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [65]:
image_target = cv2.imread("imagens/coke-video.png")
image_target_gray = cv2.cvtColor(image_target, cv2.COLOR_BGR2GRAY)

image_search = cv2.imread("imagens/coca-cola.jpg")
image_search_gray = cv2.cvtColor(image_search, cv2.COLOR_BGR2GRAY)
#image_search_gray = cv2.bitwise_not(image_search_gray)

cv2.imshow("Imagem do video", image_target_gray)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imshow("Imagen da embalagem", image_search_gray)
cv2.waitKey()
cv2.destroyAllWindows()

orb_detector = cv2.ORB_create(2000)

surf_detector = cv2.xfeatures2d.SURF_create()
surf_detector.setHessianThreshold(5000)

# Obtendo pontos e descritores da embalagem

kps = orb_detector.detect(image_target_gray, None)
kps_target, desc_target = orb_detector.compute(image_target_gray, kps)

image_detected = image_search.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow("Pontos chave da embalagem ORB", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

# Obtendo pontos e descritores do exemplo do vídeo (somente para testes)

kps = orb_detector.detect(image_search_gray, None)
kps_search, desc_search = orb_detector.compute(image_search_gray, kps)

image_detected = image_target.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow("Pontos chave da embalagem ORB", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

In [67]:
# Teste básico do algoritmo utilizando um frame do vídeo

FLANN_INDEX_LSH = 6
FLANN_INDEX_KDTREE = 0

index_params_orb = dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 12, multi_probe_level = 1) 
index_params_surf = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)

search_params = dict(checks=100)

In [71]:
flann = cv2.FlannBasedMatcher(index_params_orb, search_params)
matches = flann.knnMatch(desc_target, desc_search, k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.75*n.distance:
        matchesMask[i]=[1,0]
        
draw_params = dict(matchColor = (0,255,0), singlePointColor = (255,0,0), matchesMask = matchesMask, flags = 0)
image_detected = cv2.drawMatchesKnn(image_target, kps_target, image_search, kps_search, matches, None, **draw_params)

cv2.imshow("Logo Identificado ORB", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

In [72]:
(kps_target, desc_target) = surf_detector.detectAndCompute(image_target_gray, None)

image_detected = image_search.copy()
image_detected = cv2.drawKeypoints(image_detected, kps_target, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Pontos chave da embalagem SURF", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

(kps_search, desc_search) = surf_detector.detectAndCompute(image_search_gray, None)

image_detected = image_search.copy()
image_detected = cv2.drawKeypoints(image_detected, kps_search, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Pontos chave da embalagem SURF", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

In [73]:
# Teste básico do algoritmo utilizando um frame do vídeo

flann = cv2.FlannBasedMatcher(index_params_surf, search_params)
matches = flann.knnMatch(desc_target, desc_search, k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.9*n.distance:
        matchesMask[i]=[1,0]
        
draw_params = dict(matchColor = (0,255,0), singlePointColor = (255,0,0), matchesMask = matchesMask, flags = 0)
image_detected = cv2.drawMatchesKnn(image_target, kps_target, image_search, kps_search, matches, None, **draw_params)

cv2.imshow("Logo Identificado SURF", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

In [60]:
# Na função não vamos desenhar as correspondências de KNN, logo não é preciso construir a máscara nem utilizar o método
# de desenho destas correspondências.

# Utilize o try/catch pois as vezes os matches obtidos dos frames podem vir incompletos

def detectar_embalagem_orb(image):
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 12, multi_probe_level = 1) 
    search_params = dict(checks=100)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    image_target_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kps = orb_detector.detect(image_target_gray, None)
    kps_target, desc_target = orb_detector.compute(image_target_gray, kps)
    
    matches = flann.knnMatch(desc_target, desc_search, k=2)
        
    good_matches = []
    
    try:
        for (m,n) in matches:
            if m.distance < 0.8*n.distance:
                good_matches.append(m)
    except:
        pass
    
    return len(good_matches)

In [61]:
def detectar_embalagem_surf(image):
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)

    search_params = dict(checks=100)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    image_target_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    (kps_target, desc_target) = surf_detector.detectAndCompute(image_target_gray, None)
    
    matches = flann.knnMatch(desc_target, desc_search, k=2)
        
    good_matches = []
    
    try:
        for (m,n) in matches:
            if m.distance < 0.8*n.distance:
                good_matches.append(m)
    except:
        pass
    
    return len(good_matches)

In [63]:
import time

cam_capture = cv2.VideoCapture("videos/coke.avi")

while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
        matches = detectar_embalagem_orb(image_frame)
    
        if matches >= 100: #20 com SURF e 100 com ORB
            cv2.putText(image_frame, "Identificado " + str(matches), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 6)
    
        cv2.imshow("Sketcher ROI", image_frame)

        if cv2.waitKey(1) == 13:
            break
    else:
        break
    #time.sleep(0.01)
        
cam_capture.release()
cv2.destroyAllWindows()

## Método por câmera

In [ ]:
cam_capture.release()
cam_capture = cv2.VideoCapture(0)
cv2.destroyAllWindows()

upper_left = (50, 50)
bottom_right = (300, 300)

while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
    
        #Rectangle marker
        r = cv2.rectangle(image_frame, upper_left, bottom_right, (100, 50, 200), 5)
        rect_img = image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]]
        
        matches = detectar_embalagem_orb(rect_img)
    
        if matches > 20:
            cv2.putText(image_frame, "Identificado " + str(matches), (50,350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 6)

        cv2.imshow("Detector de Embalagem", image_frame)
        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
cam_capture.release()
cv2.destroyAllWindows()